In [ ]:
from api import LastFMApi
from scraping import GenreScraper

import requests
import pandas as pd
import time
import tqdm
import re
from bs4 import BeautifulSoup
import logging
from importlib import reload

reload(logging)
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.DEBUG, datefmt='%I:%M:%S', filename='log.log)

In [ ]:
api = LastFMApi()
scraper = GenreScraper(verbose=False)

top_world_tracks = api.get_top_chart()
top_russia_tracks = api.get_top_chart_by_country('russian federation')

all_tracks = top_world_tracks + top_russia_tracks

In [ ]:
data_for_df = []

for i in tqdm.tqdm(range(len(all_tracks))):

    track = all_tracks[i]
    track_info = api.get_track_info(track['artist'], track['track_name'])
    album_info = api.get_album_info(track['artist'], track_info['album'])
    tag_info = api.get_tag_info(track_info['toptag'])
    tag_popularity = scraper.get_genre_popularity_by_decade(track_info['toptag'], album_info['decade'], weighted=1)

    track_data = {'title': track['track_name'],
                'artist': track['artist'],
                'album': album_info['name'],
                'decade': album_info['decade'],
                'genre': track_info['toptag'],
                'listeners': track['listeners'],
                'playcount': track_info['playcount'],
                'genre_summary': tag_info['tag_summary'],
                'genre_popularity': tag_popularity}

    data_for_df.append(track_data)

In [ ]:
import json
with open('data_for_df.json', 'w') as file:
    json.dump(data_for_df, file)

In [ ]:
import pandas as pd

df = pd.DataFrame(data_for_df)
df.to_csv('data_100p.csv', index=False)